In [81]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [82]:
df = pd.read_table('Data/botswana.tsv')

In [83]:
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
ceb         4361 non-null int64
age         4361 non-null int64
educ        4361 non-null int64
religion    4361 non-null object
idlnchld    4241 non-null float64
knowmeth    4354 non-null float64
usemeth     4290 non-null float64
evermarr    4361 non-null int64
agefm       2079 non-null float64
heduc       1956 non-null float64
urban       4361 non-null int64
electric    4358 non-null float64
radio       4359 non-null float64
tv          4359 non-null float64
bicycle     4358 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 511.1+ KB


#### Число религий

In [85]:
df['religion'].nunique()

4

#### Число строк после удаления пропусков

In [86]:
df.dropna().shape[0]

1834

#### Заполнение пропусков agefm 

In [87]:
df['nevermar'] = df['agefm'].apply(lambda x: 1 if np.isnan(x) else 0)

In [88]:
df.drop(columns=['evermarr'], axis=1, inplace=True)

In [89]:
df['nevermar'].value_counts()

1    2282
0    2079
Name: nevermar, dtype: int64

In [90]:
df['agefm'] = df['agefm'].fillna(0)

In [91]:
df['heduc'].isna().sum()

2405

In [92]:
df.loc[df['nevermar'] == 1, 'heduc'] = -1

In [93]:
df['heduc'].isna().sum()

123

In [94]:
df['idlnchld_noans'] = df['idlnchld'].apply(lambda x: 1 if np.isnan(x) else 0)
df['heduc_noans'] = df['heduc'].apply(lambda x: 1 if np.isnan(x) else 0)
df['usemeth_noans'] = df['usemeth'].apply(lambda x: 1 if np.isnan(x) else 0)

In [95]:
df['idlnchld'] = df['idlnchld'].fillna(0)
df['heduc'] = df['heduc'].fillna(0)
df['usemeth'] = df['usemeth'].fillna(0)

In [96]:
df.loc[df['idlnchld_noans'] == 1, 'idlnchld'] = -1
df.loc[df['heduc_noans'] == 1, 'heduc'] = -2
df.loc[df['usemeth_noans'] == 1, 'usemeth'] = -1

In [98]:
df = df.dropna()

In [99]:
df.shape[0]

4348

### Regression model

In [100]:
' + '.join(df.columns)

'ceb + age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermar + idlnchld_noans + heduc_noans + usemeth_noans'

In [101]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + \
              agefm + heduc + urban + electric + radio + tv + bicycle + nevermar \
              + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=df)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 27 Jun 2019   Prob (F-statistic):               0.00
Time:                        23:29:37   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

### Breusch-Pagan test. Errors heterogeneus(p<=0.05)

In [109]:
print('Breusch-Pagan test: p=%s' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=1.1452927633442407e-225


In [103]:
df.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermar', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [104]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + \
              agefm + heduc + urban + electric + bicycle + nevermar \
              + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=df)
fitted2 = m2.fit(cov_type='HC1')
print(fitted2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 27 Jun 2019   Prob (F-statistic):               0.00
Time:                        23:29:38   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

### Fisher test

In [105]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


### Delete 'usemeth' and 'usemeth_noans'

In [106]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + \
              agefm + heduc + urban + electric + bicycle + nevermar \
              + idlnchld_noans + heduc_noans', 
             data=df)
fitted3 = m3.fit(cov_type='HC1')
print(fitted3.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Thu, 27 Jun 2019   Prob (F-statistic):               0.00
Time:                        23:29:39   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [108]:
print("F=%f, p=%s, k1=%f" % m2.fit().compare_f_test(m3.fit()))

F=92.890582, p=3.155200948041877e-40, k1=2.000000


### confidence intervals